# Examples using Zeep to Access SOAP Endpoints
- [References](https://docs.python-zeep.org/en/master/index.html)

In [ ]:
import json
import os
from zipfile import ZipFile

import pandas as pd
from requests import Session
from requests.auth import HTTPBasicAuth
from zeep import Client, Settings
from zeep.cache import SqliteCache
from zeep.transports import Transport

In [ ]:
%%bash

python3 -mzeep ${ERP_SERVICE_WSDL}

## Create Soap Clients
- ErpIntegrationService
- GenericSoapPort

In [ ]:
erp_uri = os.environ['ERP_SERVICE_WSDL']
soap_uri = os.environ['GENERIC_SERVICE_WSDL']
user = os.environ['FUSION_USER']
password = os.environ['FUSION_PASSWORD']

session = Session()
session.auth = HTTPBasicAuth(user, password)

settings = Settings(strict=True,
                    xml_huge_tree=True,
                    raw_response=False,
                    force_https=True
                    )
transport = Transport(cache=SqliteCache(), session=session)

client = Client(erp_uri,
                settings=settings,
                transport=transport)

soap_client = Client(soap_uri,
                     settings=settings,
                     transport=transport)

### Get Types

In [ ]:
field_element = soap_client.get_element(name='ns0:Field')
service_type = soap_client.get_type(name='ns0:Service')

## Initiate Job Submission Workflow

In [ ]:
package = "oracle/apps/ess/custom/Conversion/"
job = "BHSGLTBREP"
# package = "oracle/apps/ess/custom/Custom/"
# job = "BHSTESTUCM"

request_id = client.service.submitESSJobRequest(
    jobPackageName=package, 
    jobDefinitionName=job, 
    paramList=[
        "Primary CAD",
        "Nov-19",
        "CAD",
        "A"
    ])
request_id

In [ ]:
status = client.service.getESSJobStatus(requestId=request_id)
status

In [ ]:
details = client.service.getESSExecutionDetails(requestId=request_id)
json_details = json.loads(details)

json_details

## Handling Attachments
- [Reference](https://www.w3.org/TR/SOAP-attachments/)

In [ ]:
pack = client.service.downloadESSJobExecutionDetails(
    requestId=request_id, fileType="all")

In [ ]:
tmp_folder = 'data'

for a in pack:
    
    print(f"{a.DocumentId}, {a.DocumentName}, {a.DocumentTitle}, {a.ContentType}")
    
    tmp_zip = os.path.join(tmp_folder, a.DocumentName)
    
    with open(tmp_zip, 'wb') as f:
        f.write(a.Content)

    with ZipFile(tmp_zip) as z:
        z.extractall(path=tmp_folder)

## Issue Search
- Uses XML in CDATA section

In [ ]:
search_query = u"""
dOriginalName <starts> `MANIFEST_DATA_41`
<AND> dSecurityGroup <starts> `OBIAImport`
""".strip()

search_element = field_element(search_query, name='QueryText')

search_service = service_type(
    Document={
        "Field": [
            search_element
        ]
    },
    IdcService='GET_SEARCH_RESULTS')

response = soap_client.service.GenericSoapOperation(
    Service=search_service, webKey='cs')

for result in response['Service']['Document']['ResultSet']:
    if result['name'] == 'SearchResults':
        for row in result['Row']:
            for field in row['Field']:
                if field['name'] == 'dID':
                    document_id = field['_value_1']
                    break

if document_id:
    print(f"Found document {document_id}")

## File Download Process

### Setup search

In [ ]:
file_service = service_type(
    Document={
        "Field": [
            field_element(62962, name='dID'),
#             field_element('UCMFA00060055', name='dDocName'),
#             field_element(1, name='allowInterrupt'),
        ]
    },
    IdcService='GET_FILE')

file_service

### Issue Get File

In [ ]:
response = soap_client.service.GenericSoapOperation(
    Service=file_service, 
    webKey='cs')
response

### Get Metadata

In [ ]:
results = response['Service']['Document']['ResultSet']
data = []

for attach in results:
    if attach['name'] == 'FILE_DOC_INFO':
        for row in attach['Row']:
            record = {}
            for field in row['Field']:
                record[field['name']] = field['_value_1']
            data.append(record)

pd.DataFrame(data)

### Download Content

In [ ]:
tmp_folder = './data'

for attach in response['Service']['Document']['File']:
    with open(os.path.join(tmp_folder, attach['href']), 'wb') as f:
        f.write(attach['Contents'])
